# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,manggar,-2.8833,108.2667,24.96,89,100,3.22,ID,1737960377
1,1,waitangi,-43.9535,-176.5597,14.34,86,13,1.79,NZ,1737960378
2,2,anadyr,64.7500,177.4833,-31.72,68,75,5.00,RU,1737960379
3,3,papatowai,-46.5619,169.4708,13.90,61,34,5.02,NZ,1737960380
4,4,ushuaia,-54.8000,-68.3000,6.81,87,75,1.54,AR,1737960381


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    x="Lng",  # Longitude column
    y="Lat",  # Latitude column
    geo=True,  # Enable geographic plotting
    tiles="CartoLight",  # Base map style
    size="Humidity",  # Adjust size of points based on 'Humidity'
    color="Max Temp",  # Color points based on 'Max Temp'
    frame_width=800,
    frame_height=600,
    title="City Weather Data"
)

# Display the map
map_plot 

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Max Temp,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_cities_df = city_data_df[
    (city_data_df["Max Temp"] >= 20) &
    (city_data_df["Max Temp"] <= 30) &
    (city_data_df["Humidity"] < 60)
]


# Drop any rows with null values
filtered_cities_df = filtered_cities_df.dropna()

# Display sample data
filtered_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
24,24,basoko,1.2391,23.6160,26.40,55,85,1.68,CD,1737960403
36,36,kapenguria,1.2511,35.0895,21.57,38,3,3.71,KE,1737960418
62,62,tura,25.5198,90.2201,24.86,36,0,2.26,IN,1737960361
84,84,margaret river,-33.9500,115.0667,22.61,45,0,7.89,AU,1737960472
147,147,aketi,2.7388,23.7833,26.42,28,59,1.74,CD,1737960543


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_cities_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
24,basoko,CD,1.2391,23.6160,55,
36,kapenguria,KE,1.2511,35.0895,38,
62,tura,IN,25.5198,90.2201,36,
84,margaret river,AU,-33.9500,115.0667,45,
147,aketi,CD,2.7388,23.7833,28,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [10]:
# Set parameters to search for a hotel
radius = 5000
params = {
    "categories": "accommodation.hotel",  # Category: hotels
    "limit": 1,  # Limit results to 1 hotel
    "apiKey": geoapify_key,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
basoko - nearest hotel: No hotel found
kapenguria - nearest hotel: Kalya Hotel
tura - nearest hotel: Hotel Polo Orchid
margaret river - nearest hotel: Margaret River Hotel
aketi - nearest hotel: Hôtel Mabinza
sao vicente - nearest hotel: Pousada Vitória
alappuzha - nearest hotel: Unnis Hotel
bria - nearest hotel: No hotel found
iranshahr - nearest hotel: هتل قصِر
port augusta - nearest hotel: Hotel Commonwealth
as sulayyil - nearest hotel: No hotel found
ghanzi - nearest hotel: Ghanzi farmhouse
sur - nearest hotel: Sur Hotel
sargur - nearest hotel: No hotel found
chauk - nearest hotel: Royal Chauk
tarariras - nearest hotel: No hotel found
pokhara - nearest hotel: Hotel Annapurna View Sarangkot
kroonstad - nearest hotel: No hotel found
rangapara - nearest hotel: No hotel found
birmitrapur - nearest hotel: No hotel found
kon tum - nearest hotel: Bich Hin
lydenburg - nearest hotel: No hotel found
talamba - nearest hotel: No hotel found
pindi gheb - nearest hotel: No 

,City,Country,Lat,Lng,Humidity,Hotel Name
24,basoko,CD,1.2391,23.6160,55,No hotel found
36,kapenguria,KE,1.2511,35.0895,38,Kalya Hotel
62,tura,IN,25.5198,90.2201,36,Hotel Polo Orchid
84,margaret river,AU,-33.9500,115.0667,45,Margaret River Hotel
147,aketi,CD,2.7388,23.7833,28,Hôtel Mabinza
169,sao vicente,BR,-23.9631,-46.3919,26,Pousada Vitória
183,alappuzha,IN,9.4900,76.3264,50,Unnis Hotel
219,bria,CF,6.5423,21.9863,15,No hotel found
221,iranshahr,IR,27.2025,60.6848,21,هتل قصِر
241,port augusta,AU,-32.5000,137.7667,58,Hotel Commonwealth


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [12]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    x="Lng",  # Longitude column
    y="Lat",  # Latitude column
    geo=True,  # Enable geographic plotting
    tiles="CartoLight",  # Base map style
    size="Humidity",  # Adjust the size of points based on Humidity
    color="Hotel Name",  # Use hotel names for color categorization
    hover_cols=["City", "Country", "Hotel Name"],  # Additional hover information
    frame_width=800,
    frame_height=600,
    title="Hotels in Filtered Cities"
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,Humidity,City,Country)